In [6]:
%env CYPHER=/home/jupyter/neo4j-community-4.1.3/bin/cypher-shell
%env USER=neo4j
%env PW=neo4j

env: CYPHER=/home/jupyter/neo4j-community-4.1.3/bin/cypher-shell
env: USER=neo4j
env: PW=neo4j


In [7]:
CONNECT="$CYPHER -u $USER -p $PW"

In [8]:
!{CONNECT} "SHOW DATABASES"

+------------------------------------------------------------------------------------------------+
| name     | address          | role         | requestedStatus | currentStatus | error | default |
+------------------------------------------------------------------------------------------------+
| "neo4j"  | "localhost:7687" | "standalone" | "online"        | "online"      | ""    | TRUE    |
| "system" | "localhost:7687" | "standalone" | "online"        | "online"      | ""    | FALSE   |
+------------------------------------------------------------------------------------------------+

2 rows available after 7 ms, consumed after another 2 ms


In [10]:
!$CONNECT "MATCH (n) DETACH DELETE n"

0 rows available after 13 ms, consumed after another 0 ms


In [11]:
!$CONNECT "CALL db.indexes"
!$CONNECT "DROP INDEX ON:Listing(listing_id)"
!$CONNECT "DROP CONSTRAINT ON (a:Amenity) ASSERT a.name IS UNIQUE"
!$CONNECT "DROP CONSTRAINT ON (h:Host) ASSERT h.host_id IS UNIQUE"
!$CONNECT "DROP CONSTRAINT ON (u:User) ASSERT u.user_id IS UNIQUE"
!$CONNECT "DROP CONSTRAINT ON (n:Neighborhood) ASSERT n.neighborhood_id IS UNIQUE"

+--------------------------------------------------------------------------------------------------------------------------------------------------------------+
| id | name                  | state    | populationPercent | uniqueness  | type    | entityType | labelsOrTypes    | properties          | provider           |
+--------------------------------------------------------------------------------------------------------------------------------------------------------------+
| 5  | "constraint_3988c775" | "ONLINE" | 100.0             | "UNIQUE"    | "BTREE" | "NODE"     | ["Amenity"]      | ["name"]            | "native-btree-1.0" |
| 2  | "constraint_3e37701d" | "ONLINE" | 100.0             | "UNIQUE"    | "BTREE" | "NODE"     | ["Host"]         | ["host_id"]         | "native-btree-1.0" |
| 1  | "constraint_c0af93de" | "ONLINE" | 100.0             | "UNIQUE"    | "BTREE" | "NODE"     | ["User"]         | ["user_id"]         | "native-btree-1.0" |
| 3  | "constraint_c2c3e4d2" | "ON

In [12]:
!cat /home/jupyter/airbnb/load_data.cypher | {CONNECT} --format plain

COUNT(l)
5835
COUNT(a)
42
COUNT(n)
41
COUNT(h)
4633
COUNT(u)
55917
COUNT(r)
62976


In [13]:
!{CONNECT} "MATCH (n) RETURN COUNT(n)"

+----------+
| COUNT(n) |
+----------+
| 129444   |
+----------+

1 row available after 12 ms, consumed after another 1 ms


In [14]:
!{CONNECT} "MATCH (l:Listing) RETURN COUNT(l)"

+----------+
| COUNT(l) |
+----------+
| 5835     |
+----------+

1 row available after 14 ms, consumed after another 0 ms


In [15]:
!{CONNECT} "MATCH (a:Amenity) RETURN COUNT(a)"

+----------+
| COUNT(a) |
+----------+
| 42       |
+----------+

1 row available after 12 ms, consumed after another 0 ms


In [16]:
!{CONNECT} "MATCH (n:Neighborhood) RETURN COUNT(n)"

+----------+
| COUNT(n) |
+----------+
| 41       |
+----------+

1 row available after 13 ms, consumed after another 0 ms


In [17]:
!{CONNECT} "MATCH (h:Host) RETURN COUNT(h)"

+----------+
| COUNT(h) |
+----------+
| 4633     |
+----------+

1 row available after 14 ms, consumed after another 0 ms


In [18]:
!{CONNECT} "MATCH (u:User) RETURN COUNT(u)" 

+----------+
| COUNT(u) |
+----------+
| 55917    |
+----------+

1 row available after 9 ms, consumed after another 0 ms


In [18]:
!{CONNECT} "MATCH (r:Review) RETURN COUNT(r)"

+----------+
| COUNT(r) |
+----------+
| 62976    |
+----------+

1 row available after 8 ms, consumed after another 1 ms


## Q1

In [19]:
!{CONNECT} "MATCH (h:Host) WHERE h.location = 'Austin, Texas, United States' RETURN COUNT(h)"

+----------+
| COUNT(h) |
+----------+
| 3774     |
+----------+

1 row available after 13 ms, consumed after another 6 ms


## Q2

In [20]:
!{CONNECT} "MATCH (l:Listing)<-[:HOSTS]-(h:Host) WHERE h.host_id = '4641823' RETURN l.name, l.property_type, l.price, l.availabiliity_365 AS availability_365 ORDER BY l.price LIMIT 10"


+--------------------------------------------------------------------------------------+
| l.name                                | l.property_type | l.price | availability_365 |
+--------------------------------------------------------------------------------------+
| "1BR Convenient Austin Condo "        | "Apartment"     | 93.0    | NULL             |
| "1BR Inviting Downtown Condo, Pool"   | "Apartment"     | 99.0    | NULL             |
| "2BR/1.5BA Charming House Off SoCo"   | "House"         | 110.0   | NULL             |
| "2BR Prime East-Side Downtown"        | "House"         | 121.0   | NULL             |
| "1BR SoCo Treehouse Studio"           | "House"         | 129.0   | NULL             |
| "1BR/1.5BA East 6th, Colorful 2Story" | "Apartment"     | 134.0   | NULL             |
| "3BR Prestigious Home Travis Heights" | "House"         | 138.0   | NULL             |
| "1BR/1.5BA Perfectly Located Casita"  | "House"         | 140.0   | NULL             |
| "3BR/2BA Inviting R

## Q3

In [21]:
!{CONNECT} "MATCH (u:User)-[:WROTE]->(r:Review)-[:REVIEWS]->(l:Listing) WHERE l.listing_id = '5293632' RETURN u.id, u.name ORDER BY u.name LIMIT 10"


+--------------------------+
| u.id | u.name            |
+--------------------------+
| NULL | "Annie"           |
| NULL | "Carole"          |
| NULL | "Cory"            |
| NULL | "Craig And Trina" |
| NULL | "Dianne"          |
| NULL | "Hannah"          |
| NULL | "Jacob"           |
| NULL | "Jessie"          |
| NULL | "Johannes"        |
| NULL | "Ju-Ju"           |
+--------------------------+

10 rows available after 85 ms, consumed after another 2 ms


## Q4

In [25]:
!{CONNECT} "MATCH (u:User)-[:WROTE]->(r:Review)-[:REVIEWS]->(l:Listing)-[:HAS]->(a:Amenity) WHERE a.name CONTAINS 'Washer' AND a.name CONTAINS 'Dryer' RETURN u.id, u.name ORDER BY u.name LIMIT 10"

+------------------+
| u.id | u.name    |
+------------------+
| NULL | "Andrew"  |
| NULL | "Bryan"   |
| NULL | "Cindy"   |
| NULL | "Henrik"  |
| NULL | "Scott"   |
| NULL | "Thaniya" |
| NULL | "Warren"  |
+------------------+

7 rows available after 90 ms, consumed after another 7 ms


## Q5

In [46]:
!{CONNECT} "MATCH (l:Listing)-[:IN_NEIGHBORHOOD]->(n:Neighborhood) WHERE l.bedrooms = 3 AND n.name = 'Clarksville' RETURN l.name, l.property_type, l.price, l.availabiliity_365 AS availability_365 ORDER BY l.price LIMIT 5"


+--------------------------------------------------------------------------------------+
| l.name                                | l.property_type | l.price | availability_365 |
+--------------------------------------------------------------------------------------+
| "3BR/2.5BA Exquisite Townhouse"       | "House"         | 222.0   | NULL             |
| "3BR/2.5BA Tarrytown Duplex, Austin!" | "House"         | 249.0   | NULL             |
| "Austin downtown hideaway"            | "House"         | 249.0   | NULL             |
| "3BD Luxury Cottage by Lake Austin"   | "House"         | 290.0   | NULL             |
| "Entire Adorable Downtown House"      | "House"         | 295.0   | NULL             |
+--------------------------------------------------------------------------------------+

5 rows available after 1 ms, consumed after another 2 ms


## Q6

In [43]:
!{CONNECT} "MATCH (l:Listing)-[:HAS]->(a:Amenity) RETURN a.name, COUNT(*) AS freq ORDER BY freq DESC LIMIT 5"

+-----------------------------------+
| a.name                     | freq |
+-----------------------------------+
| "Air Conditioning"         | 5615 |
| "Wireless Internet"        | 5479 |
| "Heating"                  | 5440 |
| "Kitchen"                  | 5400 |
| "Free Parking on Premises" | 5123 |
+-----------------------------------+

5 rows available after 21 ms, consumed after another 209 ms


## Q7

In [44]:
!{CONNECT} "MATCH (n:Neighborhood)<-[:IN_NEIGHBORHOOD]-(l:Listing) RETURN n.name, n.neighborhood_id, COUNT(*) AS listing_num ORDER BY listing_num DESC LIMIT 5"


+----------------------------------------------------+
| n.name           | n.neighborhood_id | listing_num |
+----------------------------------------------------+
| NULL             | "78704"           | 1601        |
| NULL             | "78702"           | 797         |
| "Clarksville"    | "78703"           | 419         |
| "East Riverside" | "78741"           | 414         |
| NULL             | "78745"           | 328         |
+----------------------------------------------------+

5 rows available after 22 ms, consumed after another 29 ms
